In [1]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt

# === YOUR FOLDER PATHS ===
ROOT_DATA_DIR = r"C:\Users\LENOVO\OneDrive\Desktop\Project"
TEMPLATE_DIR = os.path.join(ROOT_DATA_DIR, "PCB_USED")
ALL_IMAGES_DIR = os.path.join(ROOT_DATA_DIR, "images")
OUTPUT_MASKS_DIR = os.path.join(ROOT_DATA_DIR, "PROCESSED_DEFECT_MASKS")
os.makedirs(OUTPUT_MASKS_DIR, exist_ok=True)

# Template indices (as per dataset)
TEMPLATE_INDICES = ['01', '04', '05', '06', '07', '08', '09', '10', '11', '12']
DEFECT_TYPES = ['Missing_hole', 'Mouse_bite', 'Open_circuit', 'Short', 'Spur', 'Spurious_copper']


In [2]:
def process_image_pair(template_idx, defect_type, test_img_name):
    """Loads, aligns, subtracts, and thresholds a single PCB pair."""

    # Paths
    template_path = os.path.join(TEMPLATE_DIR, f"{template_idx}.JPG")
    test_img_path = os.path.join(ALL_IMAGES_DIR, defect_type, test_img_name)

    # Read in grayscale
    template = cv2.imread(template_path, 0)
    test_img = cv2.imread(test_img_path, 0)

    if template is None or test_img is None:
        print(f"Skipping {test_img_name} — missing image.")
        return None

    # ORB feature matching for alignment
    orb = cv2.ORB_create(nfeatures=5000)
    kp1, des1 = orb.detectAndCompute(template, None)
    kp2, des2 = orb.detectAndCompute(test_img, None)

    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = sorted(bf.match(des1, des2), key=lambda x: x.distance)[:50]

    points1 = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    points2 = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

    H, _ = cv2.findHomography(points2, points1, cv2.RANSAC, 5.0)
    aligned_test = cv2.warpPerspective(test_img, H, (template.shape[1], template.shape[0]))

    # Subtract and threshold
    diff = cv2.absdiff(template, aligned_test)
    _, mask_raw = cv2.threshold(diff, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Morphological filtering
    kernel = np.ones((3, 3), np.uint8)
    mask = cv2.dilate(cv2.erode(mask_raw, kernel, 1), kernel, 1)

    return template, aligned_test, mask

In [3]:
# --- MAIN LOOP ---
for defect_type in DEFECT_TYPES:
    print(f"\nProcessing defect type: {defect_type}")
    current_dir = os.path.join(ALL_IMAGES_DIR, defect_type)
    save_dir = os.path.join(OUTPUT_MASKS_DIR, defect_type)
    os.makedirs(save_dir, exist_ok=True)

    for filename in os.listdir(current_dir):
        if not (filename.endswith(".JPG") or filename.endswith(".jpg")):
            continue

        template_idx = filename.split('_')[0]
        if template_idx not in TEMPLATE_INDICES:
            continue

        result = process_image_pair(template_idx, defect_type, filename)
        if result:
            _, _, mask = result
            mask_name = f"MASK_{filename.replace('.JPG', '.png').replace('.jpg', '.png')}"
            cv2.imwrite(os.path.join(save_dir, mask_name), mask)
            print(f"Saved: {mask_name}")

print("\n✅ All defect masks have been generated successfully!")


Processing defect type: Missing_hole
Saved: MASK_01_missing_hole_01.png
Saved: MASK_01_missing_hole_02.png
Saved: MASK_01_missing_hole_03.png
Saved: MASK_01_missing_hole_04.png
Saved: MASK_01_missing_hole_05.png
Saved: MASK_01_missing_hole_06.png
Saved: MASK_01_missing_hole_07.png
Saved: MASK_01_missing_hole_08.png
Saved: MASK_01_missing_hole_09.png
Saved: MASK_01_missing_hole_10.png
Saved: MASK_01_missing_hole_11.png
Saved: MASK_01_missing_hole_12.png
Saved: MASK_01_missing_hole_13.png
Saved: MASK_01_missing_hole_14.png
Saved: MASK_01_missing_hole_15.png
Saved: MASK_01_missing_hole_16.png
Saved: MASK_01_missing_hole_17.png
Saved: MASK_01_missing_hole_18.png
Saved: MASK_01_missing_hole_19.png
Saved: MASK_01_missing_hole_20.png
Saved: MASK_04_missing_hole_01.png
Saved: MASK_04_missing_hole_02.png
Saved: MASK_04_missing_hole_03.png
Saved: MASK_04_missing_hole_04.png
Saved: MASK_04_missing_hole_05.png
Saved: MASK_04_missing_hole_06.png
Saved: MASK_04_missing_hole_07.png
Saved: MASK_04_mi